In [3]:
import pandas as pd

In [7]:
# read df from csv
df = pd.read_csv('../data/GPT/summaries_GPT3_5_turbo_GPT4.csv')
df_BART = pd.read_csv('../data/BART/summaries_BART.csv')
df['BART_large_CNN_summary']=df_BART['extractiveSummarization']

# Compare methods

## Manual checking

In [3]:
for i in df['seniorDescription'].iloc[0:3].to_list(): print(i, '\n')

Simple Summary. This proposal calls for the integration of [Balancer Boosted Pools]([link]) into the 1inch Aggregation Protocol. Abstract. This proposal aims to grow the kinds of liquidity sources compatible with the 1inch Aggregation Protocol by integrating Balancer Boosted Pools. To fund this development, a BAL grant from Balancer DAO and Balancer DAO will be awarded to 1inch Labs upon the successful completion of the integration. Specifically, the following. 1. 1inch Labs shall integrate the Balancer Boosted Pools into the 1inch smart contracts, the 1inch dApp, and the 1inch API. 2. The integration shall be live on the Ethereum main-net no later than 3 weeks after this proposal passes the Phase-4 snapshot vote. 3. If both conditions 1 and 2 are met, Balancer DAO and Balancer DAO will transfer 20k BAL or the equivalent in stablecoins (as determined by Balancer DAO) to 1inch Labs. Motivation. Balancer Boosted Pools are a new pool type that have been gaining traction from many DeFi pro

In [4]:
for i in df['juniorDescription'].iloc[0:3].to_list(): print(i, '\n')

The proposal is about integrating Balancer Boosted Pools into the 1inch Aggregation Protocol. This would allow 1inch to have access to cheap liquidity for trade routing, and position the 1inch aggregator to be able to rapidly integrate any new Boosted Pools that are launched in the future. 

The 1inch DAO Treasury is proposing to donate 1 million USDC to support humanitarian aid within the country of Ukraine. The funds will be used to help civilians with repairing infrastructure, supporting health facilities with medicine and equipment, and supporting families with food and hygiene items. 

The Protocol Guild is a council of Ethereum protocol contributors who aim to address the challenges of attracting and retaining talent to work on the protocol. They propose that 500,000 UNI tokens (~$2.5 million at $5.19 UNI) be allocated to support the ongoing work of these core contributors in the initial Protocol Guild Pilot. 



In [5]:
for i in df['GPT4_summary'].iloc[0:3].to_list(): print(i, '\n')

This proposal suggests adding Balancer Boosted Pools to the 1inch Aggregation Protocol. Once successful, 1inch Labs will get a BAL grant. Basically, Balancer Boosted Pools will be added to 1inch’s smart contracts, app, and API. It should be done in 3 weeks after the proposal gets approved and then 1inch Labs will get 20,000 BAL or the same amount in stablecoins. The idea is to increase the range of liquidity sources and build on the popularity of Balancer Boosted Pools. If it all goes according to the plan, it will create new opportunities for 1inch and Balancer and make trading cheaper. As more Boosted Pools are created, it will grow the network. However, 1inch Labs will need to make sure everything is secure before finalizing anything. Although it’s being funded by Balancer DAO, other network updates can proceed at the same time.
 

The 1inch Network wants to donate $1 million from their DAO Treasury to support humanitarian aid in Ukraine. The DAO Treasury is a kind of savings fund t

In [6]:
for i in df_BART['extractiveSummarization'].iloc[0:3].to_list(): print(i, '\n')

Proposal aims to grow the kinds of liquidity sources compatible with the 1inch Aggregation Protocol. To fund this development, a BAL grant from Balancer DAO and Balancers DAO will be awarded to 1inch Labs. Balancer seeks to fund this integration via a payment of 20,000 BAL to 1inch Labs. The integration of Balancers BatchSwap function within the 1inch Network will also be able to be used for pools other than Boosted Pools. In the near future we expect to see meta pools created with the BPT of Boosted Pools, an example would be a bbaUSD/WETH pool. Routing trades through such pools would include one or more extra hop through Balancers BatchSwap function. This is contingent on the same proposal passing through the BalancerDAO. Support for the mentioned pools will unlock $350mm of efficient TVL for the 1inch Aggregation Protocol. Boosted Pools have been adapted by BeethovenX, Balancers Friendly Fork on Fantom and, integrating them would provide a good source for volume to 1inch. Security r

In [48]:
df[['seniorDescription', 'juniorDescription', 'GPT3_5_turbo_summary', 'GPT4_summary', 'BART_large_CNN_summary']].head(10)

,seniorDescription,juniorDescription,GPT3_5_turbo_summary,GPT4_summary,BART_large_CNN_summary
0,Simple Summary. This proposal calls for the in...,The proposal is about integrating Balancer Boo...,This proposal suggests integrating Balancer Bo...,This proposal suggests adding Balancer Boosted...,Proposal aims to grow the kinds of liquidity s...
1,Summary. 1inch Network was founded on core val...,The 1inch DAO Treasury is proposing to donate ...,This proposal suggests donating 1 million USDC...,The 1inch Network wants to donate $1 million f...,1inch Network was founded on core values of de...
2,"Authors: [Trent]([link]) (PG Member), [Tim]([l...",The Protocol Guild is a council of Ethereum pr...,This proposal suggests that the Uniswap commun...,The authors of this proposal are suggesting th...,Protocol Guild will be a vested split contract...
3,title: Aave V3 Harmony - Freeze Reserves. stat...,The proposal is about freezing all reserves on...,The proposal suggests freezing all reserves in...,This proposal by 3SE Holdings aims to address ...,The Aave DAO Community has come to the rough c...
4,Summary. 1inch is a network of decentralized p...,The proposal seeks to add 1inch Networks 1INCH...,The proposal suggests adding the 1inch Network...,"This proposal suggests onboarding 1INCH, the t...",1inch is a network of decentralized protocols ...
5,Enable agEUR as a collateral in isolation mode...,The proposal is to allow agEUR to be used as a...,The proposal suggests enabling agEUR as a coll...,"Angle Protocol wants to introduce agEUR, a Eur...",Angle Protocol is a decentralized stablecoin ...
6,[ARC] - UI/UX upgrade governance framework. Th...,The proposal is about creating a governance fr...,The proposal is to upgrade the user interface ...,"The Aave protocol, a decentralized ecosystem o...",Aave protocol is a decentralized ecosystem of ...
7,Add borrowing support for RSR (Reserve Rights ...,The Reserve team would like to add borrowing s...,The proposal is to add support for borrowing t...,The Reserve team is proposing to introduce bor...,Reserve Rights Token (RSR) is the governance t...
8,Link to proposal summary: [link] --- Requested...,The proposal is about allocating a marketing a...,The proposal suggests allocating a budget of 5...,The proposal suggests setting aside a Marketin...,"Gelato DAO has proposed a budget of 5,250,000...."
9,Proposal to vote on whether G-UNI should be sp...,The proposal is to vote on whether G-UNI shoul...,There is a proposal to vote on whether G-UNI s...,This proposal is about deciding if G-UNI shoul...,There are 2 options that can be voted on if yo...


In [7]:
# save df to csv
df[['seniorDescription', 'juniorDescription', 'GPT3_5_turbo_summary', 'GPT4_summary', 'BART_large_CNN_summary']].to_csv('../data/eval/summaries_GPT3_5_turbo_GPT4_BART.csv', index=False, header=True)


## SUPERT

In [ ]:
from metrics.SUPERT.ref_free_metrics.supert import Supert
from metrics.SUPERT.utils.data_reader import CorpusReader

# read docs and summaries
reader = CorpusReader('./metrics/SUPERT/data/topic_1')
source_docs = reader()
summaries = reader.readSummaries()

# compute the Supert scores
supert = Supert(source_docs)
scores = supert(summaries)

## SummEval

In [8]:
import os
os.environ['ROUGE_HOME'] = "./metrics/SummEval/evaluation/summ_eval/ROUGE-1.5.5"
os.environ['ROUGE_HOME']

'D:\\CVUT\\Diplom\\DIP\\metrics\\SummEval\\evaluation\\summ_eval\\ROUGE-1.5.5/'

In [9]:
from summ_eval.rouge_metric import RougeMetric
rouge = RougeMetric()

In [ ]:
summaries = ["This is one summary", "This is another summary"]
references = ["This is one reference", "This is another"]

rouge_dict = rouge.evaluate_batch(summaries, references)